# Datenbankprogrammierung

**Was ist ein Cursor? Definieren Sie das Konzept in Ihren eigenen Worten.**
  
**Aus welchem Grund (warum) und zu welchem Zweck (wozu) braucht man Cursors?**

**Wozu werden Datenbanksprachen in andere Sprachen eingebettet?**

**Was ist der Unterschied zwischen objektorientierten und objektrelationalen Datenbanken?**

**Was ist ein Surrogat?, und was hat es mit Objektorientierung zu tun?**

**Was ist das NF2 Modell? Was ist der Zusammenhang von NF2 mit der Objektorientierung?**

**Was ist objekt-relationales Mapping, und was sind die Vorteile?**


## Jupyter Setup

Um dieses Jupyter Notebook zu gebrauchen sollten folgende extensions
installiert werden:

- https://github.com/SpencerPark/IJava
- https://github.com/catherinedevlin/ipython-sql

## Datenbank Setup

Als erstes sollte [mysel installiert sein](https://support.rackspace.com/how-to/install-mysql-server-on-the-ubuntu-operating-system/).

Die Datenbank kann folgendermassen aufgesetzt werden:

```sh
sudo mysql < uni-schema.sql
sudo mysql uni4 < uni-daten.sql
```

## Hello JDBC

Professoren und Vorlesungen selektieren (ohne join)

In [1]:
%maven mysql:mysql-connector-java:8.0.19

import java.sql.Statement;
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.ResultSet;

In [2]:
Connection connection = DriverManager.getConnection(
    "jdbc:mysql://localhost:3306/uni4?&serverTimezone=Europe/Zurich",
    "uniuser", 
    "password"
);

In [3]:
var professorResultset = connection.createStatement().executeQuery("SELECT * FROM Professoren");
while (professorResultset.next()) {
    System.out.println(professorResultset.getString(2));
}

Sokrates
Russel
Kopernikus
Popper
Augustinus
Curie
Kant


In [4]:
var professorResultset = connection.createStatement().executeQuery("SELECT * FROM Professoren");
while (professorResultset.next()) {
    var statement = connection.prepareStatement("SELECT * FROM Vorlesungen WHERE gelesenVon = ?");
    statement.setInt(1, professorResultset.getInt(1));
    var vorlesungResult = statement.executeQuery();
    
    System.out.printf("%s unterrichtet: ", professorResultset.getString(2));
    while (vorlesungResult.next()) {
        System.out.printf("%s, ", vorlesungResult.getString(2));
    }
    System.out.println();
}

Sokrates unterrichtet: Logik, Ethik, Maeeutik, 
Russel unterrichtet: Erkenntnistheorie, Wissenschaftstheorie, Bioethik, 
Kopernikus unterrichtet: 
Popper unterrichtet: Der Wiener Kreis, 
Augustinus unterrichtet: Glaube und Wissen, 
Curie unterrichtet: 
Kant unterrichtet: Die 3 Kritiken, Grundzuege, 


## Exception Handling

In [9]:
connection.createStatement().execute(
    "CREATE PROCEDURE testException ()" +
    "BEGIN" +
    "    SIGNAL SQLSTATE '45000'" +
    "    SET MESSAGE_TEXT = 'An Error occurred', " +
    "    MYSQL_ERRNO = 1001;" +
    "END"
);

EvalException: PROCEDURE testException already exists

In [5]:
var resultset = connection.createStatement().executeQuery("CALL testException()");

EvalException: An Error occurred

## Stored Procedures

```sql
-- Erstellen Sie auf Ihrer lokalen MySQL-Installation eine Tabelle:
CREATE TABLE IF NOT EXISTS weekends(datum date primary key, yay boolean);

-- Dann erstellen Sie eine Stored Procedure `f`:
DELIMITER $$

CREATE PROCEDURE mark_weekends (startdate date, nr int) BEGIN

  DECLARE i int DEFAULT 1;
  DECLARE next_day date DEFAULT startdate;

  WHILE (nr >= i) DO

    -- insert weekdays and mark them as weekends
    INSERT INTO weekends(datum, yay)
    VALUES (next_day, (dayofweek(next_day) in (7, 1)));

    SET i := i + 1;
    SET next_day := DATE_ADD(next_day, INTERVAL 1 DAY);

  END WHILE;
END$$

DELIMITER ;

-- Anschliessend können Sie die die Prozedur wie folgt aufrufen:
CALL mark_weekends('2020-11-11', 111);
```

**Welche Semantik hat die Prozedur f (was genau macht das Programm)? Was bedeuten die Daten, welche die Prozedur generiert?**

Die Prozedur markiert alle Samstage und Sonntage (alle wochenenden)

In [8]:
connection.createStatement().execute("DELETE FROM weekends");
connection.createStatement().execute("CALL mark_weekends('2020-11-11', 111)");

false

In [9]:
var result = connection.createStatement().executeQuery("SELECT * FROM weekends");
while (result.next()) {
    System.out.printf("%s: %d\n", result.getString(1), result.getInt(2));
}

2020-11-11: 0
2020-11-12: 0
2020-11-13: 0
2020-11-14: 1
2020-11-15: 1
2020-11-16: 0
2020-11-17: 0
2020-11-18: 0
2020-11-19: 0
2020-11-20: 0
2020-11-21: 1
2020-11-22: 1
2020-11-23: 0
2020-11-24: 0
2020-11-25: 0
2020-11-26: 0
2020-11-27: 0
2020-11-28: 1
2020-11-29: 1
2020-11-30: 0
2020-12-01: 0
2020-12-02: 0
2020-12-03: 0
2020-12-04: 0
2020-12-05: 1
2020-12-06: 1
2020-12-07: 0
2020-12-08: 0
2020-12-09: 0
2020-12-10: 0
2020-12-11: 0
2020-12-12: 1
2020-12-13: 1
2020-12-14: 0
2020-12-15: 0
2020-12-16: 0
2020-12-17: 0
2020-12-18: 0
2020-12-19: 1
2020-12-20: 1
2020-12-21: 0
2020-12-22: 0
2020-12-23: 0
2020-12-24: 0
2020-12-25: 0
2020-12-26: 1
2020-12-27: 1
2020-12-28: 0
2020-12-29: 0
2020-12-30: 0
2020-12-31: 0
2021-01-01: 0
2021-01-02: 1
2021-01-03: 1
2021-01-04: 0
2021-01-05: 0
2021-01-06: 0
2021-01-07: 0
2021-01-08: 0
2021-01-09: 1
2021-01-10: 1
2021-01-11: 0
2021-01-12: 0
2021-01-13: 0
2021-01-14: 0
2021-01-15: 0
2021-01-16: 1
2021-01-17: 1
2021-01-18: 0
2021-01-19: 0
2021-01-20: 0
2021-0